# Preprocessing & feature Engineering

## 1. Preprocessing 
- Data shape
- Data merge
- Column prerocessing

## 2. Feature/Data Transformation
- Outlier  
- New feature

## 3. Missing Data Imputation
- Regression
- Mean

## 4. Dimensionality Reduction
- Features with Bad or Constant Data
- Multicollinearity and Variance Inflation Factor

In [2]:
%matplotlib inline
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings("ignore")
import sys
import datetime
import scipy as sp
import statsmodels.stats.api as sms
import statsmodels.api as sm
from patsy import dmatrix
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
import utils.preprocessing as pp 
import utils.correlation as cr
import utils.statsmodel_helper as st

df_macro = pd.read_csv('./github/code/data/macro.csv', parse_dates=['timestamp'])
df_train = pd.read_csv('./github/code/data/train.csv', index_col=0, parse_dates=['timestamp'])
df_test = pd.read_csv('./github/code/data/test.csv', index_col=0, parse_dates=['timestamp'])

## 1. Preprocessing 
- Data merge
- Column prerocessing

### Data merge

In [3]:
df_train.shape

(30471, 291)

In [4]:
df_train.tail(1)

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,c

In [5]:
df_test.shape

(7662, 290)

In [6]:
df_test.tail(1)

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,c

In [7]:
df_train_macro = df_train.merge(df_macro, left_on='timestamp', right_on='timestamp', how='left').set_index(df_train.index)
df_test_macro = df_test.merge(df_macro, left_on='timestamp', right_on='timestamp', how='left').set_index(df_test.index)

In [9]:
df_train_macro.shape

(30471, 390)

In [8]:
df_train_macro.tail(1)

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,c

In [10]:
df_test_macro.shape

(7662, 389)

In [11]:
df_test_macro.tail(1)

,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,preschool_quota,preschool_education_centers_raion,children_school,school_quota,school_education_centers_raion,school_education_centers_top_20_raion,hospital_beds_raion,healthcare_centers_raion,university_top_20_raion,sport_objects_raion,additional_education_raion,culture_objects_top_25,culture_objects_top_25_raion,shopping_centers_raion,office_raion,thermal_power_plant_raion,incineration_raion,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,full_all,male_f,female_f,young_all,young_male,young_female,work_all,work_male,work_female,ekder_all,ekder_male,ekder_female,0_6_all,0_6_male,0_6_female,7_14_all,7_14_male,7_14_female,0_17_all,0_17_male,0_17_female,16_29_all,16_29_male,16_29_female,0_13_all,0_13_male,0_13_female,raion_build_count_with_material_info,build_count_block,build_count_wood,build_count_frame,build_count_brick,build_count_monolith,build_count_panel,build_count_foam,build_count_slag,build_count_mix,raion_build_count_with_builddate_info,build_count_before_1920,build_count_1921-1945,build_count_1946-1970,build_count_1971-1995,build_count_after_1995,ID_metro,metro_min_avto,metro_km_avto,metro_min_walk,metro_km_walk,kindergarten_km,school_km,park_km,green_zone_km,industrial_km,water_treatment_km,cemetery_km,incineration_km,railroad_station_walk_km,railroad_station_walk_min,ID_railroad_station_walk,railroad_station_avto_km,railroad_station_avto_min,ID_railroad_station_avto,public_transport_station_km,public_transport_station_min_walk,water_km,water_1line,mkad_km,ttk_km,sadovoe_km,bulvar_ring_km,kremlin_km,big_road1_km,ID_big_road1,big_road1_1line,big_road2_km,ID_big_road2,railroad_km,railroad_1line,zd_vokzaly_avto_km,ID_railroad_terminal,bus_terminal_avto_km,ID_bus_terminal,oil_chemistry_km,nuclear_reactor_km,radiation_km,power_transmission_line_km,thermal_power_plant_km,ts_km,big_market_km,market_shop_km,fitness_km,swim_pool_km,ice_rink_km,stadium_km,basketball_km,hospice_morgue_km,detention_facility_km,public_healthcare_km,university_km,workplaces_km,shopping_centers_km,office_km,additional_education_km,preschool_km,big_church_km,church_synagogue_km,mosque_km,theater_km,museum_km,exhibition_km,catering_km,ecology,green_part_500,prom_part_500,office_count_500,office_sqm_500,trc_count_500,trc_sqm_500,cafe_count_500,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_count_500_na_price,cafe_count_500_price_500,cafe_count_500_price_1000,cafe_count_500_price_1500,cafe_count_500_price_2500,cafe_count_500_price_4000,cafe_count_500_price_high,big_church_count_500,church_count_500,mosque_count_500,leisure_count_500,sport_count_500,market_count_500,green_part_1000,prom_part_1000,office_count_1000,office_sqm_1000,trc_count_1000,trc_sqm_1000,cafe_count_1000,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_count_1000_na_price,cafe_count_1000_price_500,cafe_count_1000_price_1000,cafe_count_1000_price_1500,cafe_count_1000_price_2500,cafe_count_1000_price_4000,cafe_count_1000_price_high,big_church_count_1000,church_count_1000,mosque_count_1000,leisure_count_1000,sport_count_1000,market_count_1000,green_part_1500,prom_part_1500,office_count_1500,office_sqm_1500,trc_count_1500,trc_sqm_1500,cafe_count_1500,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_count_1500_na_price,cafe_count_1500_price_500,cafe_count_1500_price_1000,cafe_count_1500_price_1500,cafe_count_1500_price_2500,cafe_count_1500_price_4000,cafe_count_1500_price_high,big_church_count_1500,church_count_1500,mosque_count_1500,leisure_count_1500,sport_count_1500,market_count_1500,green_part_2000,prom_part_2000,office_count_2000,office_sqm_2000,trc_count_2000,trc_sqm_2000,cafe_count_2000,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,c

In [14]:
cols = list(df_train_macro.columns.values)
cols.pop(cols.index('price_doc'))
df_train_macro = df_train_macro[cols + ['price_doc']]
df_train_macro.to_csv('./github/code/data/train_macro5.csv', header=True, index=True)
df_test_macro.to_csv('./github/code/data/test_macro5.csv', header=True, index=True)

In [ ]:
df = pd.read_csv('./data/train_macro5.csv', index_col=0)
df.tail(1)

### column preprocessing

In [ ]:
# macro data에서 missing value 50% 이상이면 drop 
del df_train_macro["modern_education_share"]
del df_train_macro["old_education_build_share"]
del df_train_macro["provision_retail_space_sqm"]
del df_train_macro["provision_retail_space_modern_sqm"]
del df_train_macro["theaters_viewers_per_1000_cap"]
del df_train_macro["museum_visitis_per_100_cap"]

del df_test_macro["modern_education_share"]
del df_test_macro["old_education_build_share"]
del df_test_macro["provision_retail_space_sqm"]
del df_test_macro["provision_retail_space_modern_sqm"]
del df_test_macro["theaters_viewers_per_1000_cap"]
del df_test_macro["museum_visitis_per_100_cap"]

# odd number 45,34 etc
del df_train_macro['child_on_acc_pre_school']

# material은 카테고리
df_train_macro.loc[df_train_macro['material'].isnull(), 'material'] = 0
df_train_macro['material'] = df_train_macro['material'].replace([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0], ['a', 'b', 'c', 'd', 'e', 'f', 'e'])
df_test_macro['material'] = df_test_macro['material'].replace([0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0], ['a', 'b', 'c', 'd', 'e', 'f', 'e'])

# column rename
df_train_macro.rename(columns={'build_count_1921-1945': 'build_count_1921_1945', 'build_count_1946-1970': 'build_count_1946_1970', 'build_count_1971-1995': 'build_count_1971_1995'}, inplace=True)
df_test_macro.rename(columns={'build_count_1921-1945': 'build_count_1921_1945', 'build_count_1946-1970': 'build_count_1946_1970', 'build_count_1971-1995': 'build_count_1971_1995'}, inplace=True)


## 2. Feature/Data Transformation
- Outlier  
- New feature

## Outlier

In [ ]:
# Drop data with extremely big price #
df_train_macro = df_train_macro.drop([2121]) 

# Replace outliers with proper value #
df_train_macro.loc[df_train_macro.state == 33, 'state'] = 3
df_train_macro.loc[df_train_macro['life_sq'] > 1000,     'life_sq']       = np.median(df_train_macro['life_sq'].dropna())
df_train_macro.loc[df_train_macro['kitch_sq'] > 250,     'kitch_sq']      = np.median(df_train_macro['kitch_sq'].dropna())
df_train_macro.loc[df_train_macro['num_room'] > 6,       'num_room']      = np.median(df_train_macro['num_room'].dropna())
df_train_macro.loc[df_train_macro['build_year'] > 2017,  'build_year']    = np.median(df_train_macro['build_year'].dropna())
df_train_macro.loc[df_train_macro['build_year'] < 1800,  'build_year']    = np.median(df_train_macro['build_year'].dropna())
df_train_macro.loc[df_train_macro['floor'] > 50,         'floor']         = np.median(df_train_macro['floor'].dropna())
df_train_macro.loc[df_train_macro['max_floor'] > 60,     'max_floor']     = np.median(df_train_macro['max_floor'].dropna())
df_train_macro.loc[df_train_macro.full_sq == 0, 'full_sq'] = 50
df_train_macro = df_train_macro[df_train_macro.price_doc/df_train_macro.full_sq <= 600000]
df_train_macro = df_train_macro[df_train_macro.price_doc/df_train_macro.full_sq >= 10000]

df_test_macro.loc[df_test_macro['life_sq'] > 1000,     'life_sq']       = np.median(df_test_macro['life_sq'].dropna())
df_test_macro.loc[df_test_macro['kitch_sq'] > 250,     'kitch_sq']      = np.median(df_test_macro['kitch_sq'].dropna())
df_test_macro.loc[df_test_macro['num_room'] > 6,       'num_room']      = np.median(df_test_macro['num_room'].dropna())
df_test_macro.loc[df_test_macro['build_year'] > 2017,  'build_year']    = np.median(df_test_macro['build_year'].dropna())
df_test_macro.loc[df_test_macro['build_year'] < 1800,  'build_year']    = np.median(df_test_macro['build_year'].dropna())
df_test_macro.loc[df_test_macro['floor'] > 50,         'floor']         = np.median(df_test_macro['floor'].dropna())
df_test_macro.loc[df_test_macro['max_floor'] > 60,     'max_floor']     = np.median(df_test_macro['max_floor'].dropna())
df_test_macro.loc[df_test_macro.full_sq == 0, 'full_sq'] = 50

## New features

In [ ]:
# Add month and day of week #
df_train_macro['month'] = df_train_macro.timestamp.dt.month
df_train_macro['dow'] = df_train_macro.timestamp.dt.dayofweek

df_test_macro['month'] = df_test_macro.timestamp.dt.month
df_test_macro['dow'] = df_test_macro.timestamp.dt.dayofweek

# Create new features that might help #
df_train_macro['rel_floor'] = df_train_macro['floor'] / df_train_macro['max_floor'].astype(float)
df_train_macro['rel_kitch_sq'] = df_train_macro['kitch_sq'] / df_train_macro['full_sq'].astype(float)

df_test_macro['rel_floor'] = df_test_macro['floor'] / df_test_macro['max_floor'].astype(float)
df_test_macro['rel_kitch_sq'] = df_test_macro['kitch_sq'] / df_test_macro['full_sq'].astype(float)

df_train_macro.apartment_name=df_train_macro.sub_area + df_train_macro['metro_km_avto'].astype(str)
df_test_macro.apartment_name=df_test_macro.sub_area + df_test_macro['metro_km_avto'].astype(str)

df_train_macro['room_size'] = df_train_macro['life_sq'] / df_train_macro['num_room'].astype(float)
df_test_macro['room_size'] = df_test_macro['life_sq'] / df_test_macro['num_room'].astype(float)

# Average price corresponding to sub_area and ID_* #
id_features = ['ID_metro',
    'ID_railroad_station_walk', \
    'ID_big_road1', \
    'ID_big_road2', \
    'ID_railroad_terminal', \
    'ID_bus_terminal']

for id_f in id_features:
    df_test_macro['avg_price_' + id_f] = 0.0
    for val in df_test_macro[id_f].unique():
        if val == 171 and id_f == 'ID_metro':
            df_test_macro.loc[df_test_macro.ID_metro == 171, 'avg_price_ID_metro'] = df_train_macro[df_train_macro.ID_metro == 170]['price_doc'].mean()
            continue
        if val == 132 and id_f == 'ID_railroad_station_walk':
            df_test_macro.loc[df_test_macro.ID_railroad_station_walk == 132, 'avg_price_ID_railroad_station_walk'] = df_train_macro[df_train_macro.ID_railroad_station_walk == 131]['price_doc'].mean()
            continue
        if val == 121 and id_f == 'ID_railroad_station_walk':
            df_test_macro.loc[df_test_macro.ID_railroad_station_walk == 122, 'avg_price_ID_railroad_station_walk'] = df_train_macro[df_train_macro.ID_railroad_station_walk == 131]['price_doc'].mean()
            continue
        avg = df_train_macro[df_train_macro[id_f] == val]['price_doc'].mean()
        df_test_macro.loc[df_test_macro[id_f] == val, 'avg_price_' + id_f] = avg
    del df_test_macro[id_f]
    
for id_f in id_features:
    df_train_macro['avg_price_' + id_f] = 0.0
    for val in df_train_macro[id_f].unique():
        avg = df_train_macro[df_train_macro[id_f] == val]['price_doc'].mean()
        df_train_macro.loc[df_train_macro[id_f] == val, 'avg_price_' + id_f] = avg
    del df_train_macro[id_f]
    
cols = list(df_train_macro.columns.values)
cols.pop(cols.index('price_doc'))
df_train_macro = df_train_macro[cols + ['price_doc']]


df_test_macro['avg_price_sub_area'] = 0.0
df_train_macro['avg_price_sub_area'] = 0.0
for subarea in df_train_macro['sub_area'].unique():
    avg = df_train_macro[df_train_macro['sub_area'] == subarea]['price_doc'].mean()
    df_train_macro.loc[df_train_macro['sub_area'] == subarea, 'avg_price_sub_area'] = avg
    df_test_macro.loc[df_test_macro['sub_area'] == subarea, 'avg_price_sub_area'] = avg
del df_train_macro['sub_area']
del df_test_macro['sub_area']

In [ ]:
cols = list(df_train_macro.columns.values)
cols.pop(cols.index('price_doc'))
df_train_macro = df_train_macro[cols + ['price_doc']]
df_train_macro.to_csv('./data/train_macro6.csv', header=True, index=True)
df_test_macro.to_csv('./data/test_macro6.csv', header=True, index=True)

## 3. Missing Data Imputation
- Regression
- Mean

### Regression

In [ ]:
pp.find_missing_data_columns(df_train_macro)

In [ ]:
pp.find_missing_data_columns(df_test_macro)

In [ ]:
df_train_macro.replace([np.inf, -np.inf], np.nan, inplace=True)
df_test_macro.replace([np.inf, -np.inf], np.nan, inplace=True)
df_train_macro._get_numeric_data()[df_train._get_numeric_data() < 0] = 0
df_test_macro._get_numeric_data()[df_test._get_numeric_data() < 0] = 0

In [ ]:
pp.impute_by_regression(df_train_macro, 4, 0.3)
pp.impute_by_regression2(df_test_macro, 4, 0.3)

# imputation
pp.impute_num_mode(df_train_macro)
pp.impute_num_mode(df_test_macro)
# pp.imput_cat_mode(df_train_macro)
# pp.imput_cat_mode(df_test_macro)

In [ ]:
# Delete many nan columns
df_missing = df_test_macro.isnull().sum(axis=0).reset_index()
df_missing.columns = ['column_name', 'missing_count']
df_missing = df_missing.loc[df_missing['missing_count']>0]
nan_col = list(df_missing['column_name'])
nan_col.pop(nan_col.index('product_type'))

# nan_col = [
#  'grp',
#  'grp_growth',
#  'real_dispos_income_per_cap_growth',
#  'profitable_enterpr_share',
#  'unprofitable_enterpr_share',
#  'share_own_revenues',
#  'overdue_wages_per_cap',
#  'fin_res_per_cap',
#  'marriages_per_1000_cap',
#  'divorce_rate',
#  'construction_value',
#  'invest_fixed_assets_phys',
#  'pop_migration',
#  'pop_total_inc',
#  'housing_fund_sqm',
#  'lodging_sqm_per_cap',
#  'water_pipes_share',
#  'baths_share',
#  'sewerage_share',
#  'gas_share',
#  'hot_water_share',
#  'electric_stove_share',
#  'heating_share',
#  'old_house_share',
#  'infant_mortarity_per_1000_cap',
#  'perinatal_mort_per_1000_cap',
#  'incidence_population',
#  'load_of_teachers_preschool_per_teacher',
#  'child_on_acc_pre_school',
#  'provision_doctors',
#  'power_clinics',
#  'hospital_beds_available_per_cap',
#  'hospital_bed_occupancy_per_year',
#  'population_reg_sports_share',
#  'students_reg_sports_share',
#  'apartment_build'
# ]

for col in nan_col:
    if col in df_test_macro:
        del df_test_macro[col]
        
for col in nan_col:
    if col in df_train_macro:
        del df_train_macro[col]       

In [ ]:
cols = list(df_train_macro.columns.values)
cols.pop(cols.index('price_doc'))
df_train_macro = df_train_macro[cols + ['price_doc']]
df_train_macro.to_csv('./data/train_macro6.csv', header=True, index=True)
df_test_macro.to_csv('./data/test_macro6.csv', header=True, index=True)